This notebook is aiming at analyzing keyword frequency in training data. It generates heatmap for each class, in each heatmap, Y-axis shows each keyword from this class, and X-axis shows all the training classes. Each numerical value in the heatmap shows the number of records this keyword appears in corresponding class's training data. (If it appears multiple times in one text file, it will only be counted 1 in the heatmap)

Pre-requisite:

1) An csv file containing all the keywords and classes information, prepared in training fotmat

2) Prepared training data

Output:

1) Heatmaps for each class, containing keyword frequency information

In [1]:
# Import required libraires
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Load keyword excel
keyword_file = "keywords_to_training.csv"
keyword_df = pd.read_csv(keyword_file)
keyword_df

class                        content
0                admit_type            inpatient admission
1                admit_type             admit to inpatient
2                admit_type             admit as inpatient
3                admit_type      admission for observation
4                admit_type               same day surgery
..                      ...                            ...
578  nutritional_assessment      nutritional re-assessment
579  nutritional_assessment             nutrition services
580  nutritional_assessment        malnutrition assessment
581  nutritional_assessment  nutrition evaluation/planning
582  nutritional_assessment            nutrition documents

[583 rows x 2 columns]

In [4]:
# Load training records
training_df = pd.read_csv('batch3_combined_multi_label.csv')
training_df

content  \
0       !! UNABLE TO FIND 4 tablets every morning. Med...   
1       !! predniSONE 20 MG tabletTake 40mg of prednis...   
2       !06$69608UPS Restrictions00000200 NOSLS LISIA ...   
3       !1Authorization is hereby given to dispense th...   
4       !I, AVILA, ANIBAL, and/or designee, have recei...   
...                                                   ...   
832117  © 2000-2021 The StayWell Company, LLC. All rig...   
832118  © 2016, Elsevier/Gold Standard. (2015-11-18 15...   
832119  àPage: 173PIERCE, HENRY CHARLESFac: St. Joseph...   
832120  àPage: 323PIERCE, HENRY CHARLESFac: St. Joseph...   
832121  ⑇~1356072~51260774~I&O_RPT~##ENHMC VRF ReportL...   

                                                    class  
0       admission_reason|labs|admission_order|discharg...  
1                                       discharge_summary  
2                                                  others  
3                                                  others  
4                                                  others  
...                                                   ...  
832117                                             others  
832118                                             others  
832119                                             others  
832120                                             others  
832121                                             others  

[832122 rows x 2 columns]

In [16]:
# Create mappings from class name to numerical value
mapping_dict = {'admit_type' : 0, 'admission_reason' : 1, 'admission_order' : 2, 'mar' : 3, 'flowsheets' : 4,
               'progress_notes' : 5, 'emergency_department' : 6, 'consult' : 7, 'procedures' : 8, 
               'physician_query' : 9, 'radiology' : 10, 'discharge_summary': 11, 'history_physical': 12, 'labs' : 13,
               'case_management' : 14, 'discharge_disposition' : 15, 'nutritional_assessment' : 16, 'others' : 17}

##### Example to generate heatmap just for "admit_type"

In [17]:
# Decide class and create heatmap matrix
current_class = 'admit_type'
sub_matrix = keyword_df[keyword_df['class']== current_class]
matrix_res = np.zeros((len(sub_matrix), 1 + len(keyword_df['class'].unique())))
matrix_res.shape

(29, 18)

In [18]:
# Main part to check if keyword is in a training record
corres_index = mapping_dict[current_class]
for i in range(len(sub_matrix)):
    keyword_content = sub_matrix['content'].iloc[i]
    for row_index in range(len(training_df)):
        row_content = training_df['content'].iloc[row_index]
        row_class = training_df['class'].iloc[row_index]
        # Check if keywords are in the text 
        if keyword_content in row_content:
            # Check if keyword are in correct category
            if current_class in row_class:
                matrix_res[i, corres_index] = matrix_res[i, corres_index] + 1
            else:
                # if not in correct class, just document the first class
                first_class = row_class.split('|')[0]
                matrix_res[i, mapping_dict[first_class]] = matrix_res[i, mapping_dict[first_class]] + 1

In [26]:
# Covert heatmap from float to int
matrix_res = matrix_res.astype(int)

In [37]:
# Generate heatmap for analysis
fig, ax = plt.subplots (figsize= (15,15))
sns.set(font_scale=1)
heatmap = sns.heatmap(matrix_res, annot=True, fmt='d', cbar=False)

heatmap.yaxis.set_ticklabels(sub_matrix['content'].to_list(), rotation=0, fontsize='10', va='center')
heatmap.xaxis.set_ticklabels(mapping_dict.keys(), rotation=45, fontsize='10', va='center')

plt.savefig(f'keyword_analysis/{current_class}.png')

Above parts are for generating example

===============================================

###### Main code to generate analysis for every class, and save generated heatmap in folder

In [ ]:
for current_class in keyword_df['class'].unique():
    print(current_class)
    sub_matrix = keyword_df[keyword_df['class']== current_class]
    matrix_res = np.zeros((len(sub_matrix), 1 + len(keyword_df['class'].unique())))
    
    corres_index = mapping_dict[current_class]
    for i in range(len(sub_matrix)):
        keyword_content = sub_matrix['content'].iloc[i]
        for row_index in range(len(training_df)):
            row_content = training_df['content'].iloc[row_index]
            row_class = training_df['class'].iloc[row_index]
            # Check if keywords are in the text 
            if keyword_content in row_content:
                # Check if keyword are in correct category
                if current_class in row_class:
                    matrix_res[i, corres_index] = matrix_res[i, corres_index] + 1
                else:
                    # if not in correct class, just document the first class
                    first_class = row_class.split('|')[0]
                    matrix_res[i, mapping_dict[first_class]] = matrix_res[i, mapping_dict[first_class]] + 1    
                    
    matrix_res = matrix_res.astype(int)

    fig, ax = plt.subplots (figsize= (15,15))
    sns.set(font_scale=1)
    heatmap = sns.heatmap(matrix_res, annot=True, fmt='d', cbar=False)

    heatmap.yaxis.set_ticklabels(sub_matrix['content'].to_list(), rotation=0, fontsize='10', va='center')
    heatmap.xaxis.set_ticklabels(mapping_dict.keys(), rotation=45, fontsize='10', va='center')

    plt.savefig(f'keyword_analysis/{current_class}.png')